### **Feauture Extraction (1st phase)** **bold text**

Model

In [3]:
import tensorflow
from tensorflow.keras.preprocessing import image                                     # converting images to arrays, resizing, etc.
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))   # Loading the ResNet50 model with pre-trained ImageNet weights, but without the top (classification) layer.
                                                                                 # We’ll use it just for feature extraction, not training.
model.trainable = False                                                          # Freezing the model so its weights don’t change during training

model = tensorflow.keras.Sequential([model,GlobalMaxPooling2D()])                # Creating a new model by stacking the frozen ResNet50 and a GlobalMaxPooling layer to flatten the output.

print(model.summary())                                                           # Printing the model’s structure to see the layers and output shapes.



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling2d                 │ (None, 2048)                │               0 │
│ (GlobalMaxPooling2D)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

None


Function for extracting features

In [2]:
import numpy as np
from numpy.linalg import norm


In [3]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

Accessing the dataset

In [9]:
!unzip "/content/images.zip" -d /content/dataset


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/images/58128.jpg  
  inflating: /content/dataset/images/58129.jpg  
  inflating: /content/dataset/images/5813.jpg  
  inflating: /content/dataset/images/58131.jpg  
  inflating: /content/dataset/images/58132.jpg  
  inflating: /content/dataset/images/58133.jpg  
  inflating: /content/dataset/images/58135.jpg  
  inflating: /content/dataset/images/58136.jpg  
  inflating: /content/dataset/images/58137.jpg  
  inflating: /content/dataset/images/58138.jpg  
  inflating: /content/dataset/images/58139.jpg  
  inflating: /content/dataset/images/5814.jpg  
  inflating: /content/dataset/images/58140.jpg  
  inflating: /content/dataset/images/58141.jpg  
  inflating: /content/dataset/images/58143.jpg  
  inflating: /content/dataset/images/58144.jpg  
  inflating: /content/dataset/images/58145.jpg  
  inflating: /content/dataset/images/58146.jpg  
  inflating: /content/dataset/images/58147.jpg  
  inflating: /conten

Getting the paths of the images in the dataset

In [10]:
import os

filenames = []
for filename in os.listdir("/content/dataset/images"):
    filenames.append(os.path.join("/content/dataset/images", filename))

print(len(filenames))
print(filenames[0:5])

44441
['/content/dataset/images/36753.jpg', '/content/dataset/images/47376.jpg', '/content/dataset/images/3011.jpg', '/content/dataset/images/16819.jpg', '/content/dataset/images/22741.jpg']


Creating a feature list (2D array) for all 44K images with 2048 features each

In [2]:
from tqdm import tqdm

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

# Storing the feautures in a file for further use
import pickle

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))

NameError: name 'filenames' is not defined

In [1]:
# Storing the feautures in a file for further use
import pickle

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))

NameError: name 'feature_list' is not defined

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
